<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#groupBy" data-toc-modified-id="groupBy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>groupBy</a></span><ul class="toc-item"><li><span><a href="#rollup" data-toc-modified-id="rollup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>rollup</a></span></li><li><span><a href="#cube" data-toc-modified-id="cube-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>cube</a></span></li></ul></li><li><span><a href="#pivot" data-toc-modified-id="pivot-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>pivot</a></span></li></ul></div>

In [10]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [11]:
spark

In [12]:
from pyspark.sql.functions import *

meteo = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('../donnees/meteo30')\
    .cache()

meteo = meteo.select(
                 col('numer_sta'),
                 to_timestamp(col('date').cast('string'),'yyyyMMddHHmmss'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','date','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()


from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('../donnees/meteo/postesSynop.csv')  \
      .cache()

meteo.count(), villes.count()

(4102262, 62)

# groupBy

<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-03.png" width="400">    

In [13]:
meteo.where('id < 8000')\
     .select('annee','mois_jour','temperature','precipitations')\
     .describe().show()

+-------+------------------+-----------------+------------------+--------------------+
|summary|             annee|        mois_jour|       temperature|      precipitations|
+-------+------------------+-----------------+------------------+--------------------+
|  count|           2924329|          2924329|           2917134|             2911157|
|   mean|2008.8013773416055|667.2736111429323|12.688548061213664|  0.2606704224471589|
| stddev| 7.073323663064156| 342.752779649345| 7.327212877431764|  1.3221662599496276|
|    min|              1997|             0101|             -21.3|-0.30000000000000004|
|    max|              2021|             1231|              46.9|               443.0|
+-------+------------------+-----------------+------------------+--------------------+



In [14]:
meteo.where('id < 8000')\
     .select('annee','mois_jour','temperature')\
     .describe().show()

+-------+------------------+-----------------+------------------+
|summary|             annee|        mois_jour|       temperature|
+-------+------------------+-----------------+------------------+
|  count|           2924329|          2924329|           2917134|
|   mean|2008.8013773416055|667.2736111429323|12.688548061213664|
| stddev| 7.073323663064156| 342.752779649345| 7.327212877431764|
|    min|              1997|             0101|             -21.3|
|    max|              2021|             1231|              46.9|
+-------+------------------+-----------------+------------------+



In [15]:
meteo.where('id < 8000').count()

2924329

In [16]:
meteo.where('id < 8000')\
     .select('humidite','visibilite','pression')\
     .describe().show()

+-------+-------------------+------------------+------------------+
|summary|           humidite|        visibilite|          pression|
+-------+-------------------+------------------+------------------+
|  count|            2914795|           2524195|           2911657|
|   mean| 0.7553112791808771|22.380299168646044| 99.56710474482422|
| stddev|0.17687823174673353| 15.13196338966417|2.5289770316208258|
|    min|               0.01|               0.0|              43.4|
|    max|                1.0|             80.01|            105.09|
+-------+-------------------+------------------+------------------+



In [17]:
meteo.where('id < 8000')\
     .groupBy('annee')\
     .avg('temperature','visibilite','pression').show(5)

+-----+------------------+------------------+-----------------+
|annee|  avg(temperature)|   avg(visibilite)|    avg(pression)|
+-----+------------------+------------------+-----------------+
| 2018| 13.37747716669949|24.960805049257843|  99.493988174803|
| 2019|13.185722746378648|25.786389970746466|99.53540068658691|
| 2020|15.308303801996642|27.542648986820424|99.51710724780125|
| 2017|12.851390398103337|25.692301466912546|99.75424086095852|
| 2021|12.864146372890076| 25.19687902899523|99.60843868757007|
+-----+------------------+------------------+-----------------+
only showing top 5 rows



In [18]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .max('temperature','visibilite','pression').show(5)

+----+-----+----------------+---------------+-------------+
|  id|annee|max(temperature)|max(visibilite)|max(pression)|
+----+-----+----------------+---------------+-------------+
|7747| 2020|            36.2|           60.0|        102.8|
|7790| 2018|            33.7|           60.0|       103.41|
|7181| 2018|            35.2|           60.0|        99.58|
|7558| 2019|            36.3|           20.0|        94.96|
|7558| 2018|            34.1|           20.0|        95.33|
+----+-----+----------------+---------------+-------------+
only showing top 5 rows



In [19]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .agg(
            count('id').alias('nb_villes'),
            round(avg('temperature'),2).alias('temperature'),
            round(avg('humidite'),2).alias('humidite'),
            round(avg('visibilite'),2).alias('visibilite'),
            round(avg('pression'),2).alias('pression'),
            round(sum('pression')).alias('precipitations'))\
     .orderBy("id","annee")\
     .show(28)

+----+-----+---------+-----------+--------+----------+--------+--------------+
|  id|annee|nb_villes|temperature|humidite|visibilite|pression|precipitations|
+----+-----+---------+-----------+--------+----------+--------+--------------+
|7005| 1997|     2904|      10.54|    0.83|     12.91|   100.8|      292736.0|
|7005| 1998|     2912|      10.55|    0.83|     14.52|   100.7|      293250.0|
|7005| 1999|     2902|      11.14|    0.82|     13.85|   100.6|      291927.0|
|7005| 2000|     2903|      10.94|    0.84|     14.01|  100.57|      291960.0|
|7005| 2001|     2882|       10.6|    0.84|     14.87|  100.66|      290096.0|
|7005| 2002|     2894|      10.95|    0.83|     16.48|  100.61|      290059.0|
|7005| 2003|     2905|      10.93|    0.79|     16.98|  100.83|      292904.0|
|7005| 2004|     2911|      10.71|    0.83|     17.87|  100.75|      293285.0|
|7005| 2005|     2879|      10.79|    0.84|     17.36|  100.89|      290469.0|
|7005| 2006|     2909|      11.08|    0.83|     17.5

In [20]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .orderBy('annee','mois')\
     .show(5)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 1997|   1|         982.0|
| 1997|   2|         890.0|
| 1997|   3|         985.0|
| 1997|   4|         951.0|
| 1997|   5|        1003.0|
+-----+----+--------------+
only showing top 5 rows



## rollup
<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-05.png" width="400">

In [21]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .rollup('annee','mois')\
     .agg( round(sum('precipitations')).alias('precipitations'))\
     .orderBy(col('annee').asc_nulls_last(),
              col('mois').asc_nulls_last())\
     .show(14)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 1997|   1|         982.0|
| 1997|   2|         890.0|
| 1997|   3|         985.0|
| 1997|   4|         951.0|
| 1997|   5|        1003.0|
| 1997|   6|         963.0|
| 1997|   7|         993.0|
| 1997|   8|         989.0|
| 1997|   9|         972.0|
| 1997|  10|         991.0|
| 1997|  11|         957.0|
| 1997|  12|         992.0|
| 1997|null|       11668.0|
| 1998|   1|         997.0|
+-----+----+--------------+
only showing top 14 rows



In [22]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .orderBy('annee','mois')\
     .show()

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 1997|   1|         982.0|
| 1997|   2|         890.0|
| 1997|   3|         985.0|
| 1997|   4|         951.0|
| 1997|   5|        1003.0|
| 1997|   6|         963.0|
| 1997|   7|         993.0|
| 1997|   8|         989.0|
| 1997|   9|         972.0|
| 1997|  10|         991.0|
| 1997|  11|         957.0|
| 1997|  12|         992.0|
| 1998|   1|         997.0|
| 1998|   2|         916.0|
| 1998|   3|        1011.0|
| 1998|   4|         955.0|
| 1998|   5|        1001.0|
| 1998|   6|         971.0|
| 1998|   7|        1002.0|
| 1998|   8|        1001.0|
+-----+----+--------------+
only showing top 20 rows



In [23]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .rollup('annee','mois')\
     .agg( round(sum('precipitations')).alias('precipitations'))\
     .orderBy('annee','mois')\
     .show()

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| null|null|      289907.0|
| 1997|null|       11668.0|
| 1997|   1|         982.0|
| 1997|   2|         890.0|
| 1997|   3|         985.0|
| 1997|   4|         951.0|
| 1997|   5|        1003.0|
| 1997|   6|         963.0|
| 1997|   7|         993.0|
| 1997|   8|         989.0|
| 1997|   9|         972.0|
| 1997|  10|         991.0|
| 1997|  11|         957.0|
| 1997|  12|         992.0|
| 1998|null|       11796.0|
| 1998|   1|         997.0|
| 1998|   2|         916.0|
| 1998|   3|        1011.0|
| 1998|   4|         955.0|
| 1998|   5|        1001.0|
+-----+----+--------------+
only showing top 20 rows



In [24]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .rollup('annee','mois')\
     .agg(round(sum('precipitations')).alias('precipitations'))\
     .orderBy('annee','mois')\
     .show(20)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| null|null|      289907.0|
| 1997|null|       11668.0|
| 1997|   1|         982.0|
| 1997|   2|         890.0|
| 1997|   3|         985.0|
| 1997|   4|         951.0|
| 1997|   5|        1003.0|
| 1997|   6|         963.0|
| 1997|   7|         993.0|
| 1997|   8|         989.0|
| 1997|   9|         972.0|
| 1997|  10|         991.0|
| 1997|  11|         957.0|
| 1997|  12|         992.0|
| 1998|null|       11796.0|
| 1998|   1|         997.0|
| 1998|   2|         916.0|
| 1998|   3|        1011.0|
| 1998|   4|         955.0|
| 1998|   5|        1001.0|
+-----+----+--------------+
only showing top 20 rows



In [25]:
meteo.where('id < 8000')\
     .rollup('annee','mois')\
     .agg(
            round(sum('precipitations')).alias('precipitations'))\
     .orderBy(col('annee').asc_nulls_last(),
              col('mois').asc_nulls_last())\
     .show(16)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 1997|   1|        1988.0|
| 1997|   2|        2242.0|
| 1997|   3|         421.0|
| 1997|   4|         908.0|
| 1997|   5|        2980.0|
| 1997|   6|        4786.0|
| 1997|   7|        1816.0|
| 1997|   8|        2988.0|
| 1997|   9|         821.0|
| 1997|  10|        2386.0|
| 1997|  11|        4664.0|
| 1997|  12|        3854.0|
| 1997|null|       29854.0|
| 1998|   1|        3178.0|
| 1998|   2|         776.0|
| 1998|   3|        1409.0|
+-----+----+--------------+
only showing top 16 rows



In [26]:
meteo.where('id < 8000')\
     .rollup('annee','mois')\
     .agg(
            round(avg('temperature'),2).alias('temperature'),
            round(avg('humidite'),2).alias('humidite'),
            round(avg('visibilite'),2).alias('visibilite'),
            round(avg('pression'),2).alias('pression'),
            round(avg('precipitations'),2).alias('precipitations'))\
     .orderBy(col('annee').asc_nulls_last(),
              col('mois').asc_nulls_last())\
     .show(16)

+-----+----+-----------+--------+----------+--------+--------------+
|annee|mois|temperature|humidite|visibilite|pression|precipitations|
+-----+----+-----------+--------+----------+--------+--------------+
| 1997|   1|       3.26|    0.86|      9.56|   99.74|          0.23|
| 1997|   2|       7.88|    0.79|     15.39|  100.19|          0.28|
| 1997|   3|       10.2|    0.73|     16.62|  100.28|          0.04|
| 1997|   4|       11.0|    0.62|     20.96|   99.75|          0.09|
| 1997|   5|      15.11|    0.71|     20.51|   99.33|           0.3|
| 1997|   6|      17.09|    0.76|     19.18|   98.99|          0.49|
| 1997|   7|      19.15|    0.74|     19.84|   99.72|          0.18|
| 1997|   8|      21.76|    0.75|     17.97|    99.5|           0.3|
| 1997|   9|      17.63|    0.74|     18.51|   99.92|          0.08|
| 1997|  10|      13.24|    0.78|     17.16|   99.54|          0.24|
| 1997|  11|        9.4|    0.85|     15.21|   98.65|          0.48|
| 1997|  12|       6.51|    0.84| 

## cube
<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-06.png" width="400">

In [27]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .cube('annee','mois')\
     .agg( round(sum('precipitations')).alias('precipitations'))\
     .orderBy(col('annee'),col('mois'))\
     .show(24)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| null|null|      289907.0|
| null|   1|       24263.0|
| null|   2|       22070.0|
| null|   3|       24111.0|
| null|   4|       24339.0|
| null|   5|       25200.0|
| null|   6|       24365.0|
| null|   7|       25163.0|
| null|   8|       25176.0|
| null|   9|       23362.0|
| null|  10|       24163.0|
| null|  11|       23411.0|
| null|  12|       24284.0|
| 1997|null|       11668.0|
| 1997|   1|         982.0|
| 1997|   2|         890.0|
| 1997|   3|         985.0|
| 1997|   4|         951.0|
| 1997|   5|        1003.0|
| 1997|   6|         963.0|
| 1997|   7|         993.0|
| 1997|   8|         989.0|
| 1997|   9|         972.0|
| 1997|  10|         991.0|
+-----+----+--------------+
only showing top 24 rows



In [28]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .agg(
            {'id':'count',
            'temperature':'avg',
            'humidite':'avg'}
     ).show(10)

+----+-----+------------------+------------------+---------+
|  id|annee|     avg(humidite)|  avg(temperature)|count(id)|
+----+-----+------------------+------------------+---------+
|7747| 2020|0.6630913242009133|18.260542030316955|     2199|
|7790| 2018|0.7473152286008939|16.811172224132005|     2916|
|7181| 2018|0.7558756868131872|11.667949158364825|     2912|
|7558| 2019|0.7018670886075954|11.308966244725742|     2844|
|7558| 2018| 0.739443099273608|11.813490141819441|     2891|
|7481| 2019|0.6953703703703707|13.503806584362135|     2916|
|7607| 2020|0.7647740757644912|16.179141944317678|     2197|
|7139| 2020|0.7439328426862939|14.197884084636618|     2175|
|7181| 2020|0.6911885245901641|14.058105646630224|     2197|
|7020| 2018|0.8082445987654319|12.319087452471484|     2840|
+----+-----+------------------+------------------+---------+
only showing top 10 rows



In [29]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .agg(
            {'id':'count',
            'temperature':'avg',
            'humidite':'avg'}
     ).toDF('id','annee','humidite','temperature','nb_villes').show(10)

+----+-----+------------------+------------------+---------+
|  id|annee|          humidite|       temperature|nb_villes|
+----+-----+------------------+------------------+---------+
|7747| 2020|0.6630913242009133|18.260542030316955|     2199|
|7790| 2018|0.7473152286008939|16.811172224132005|     2916|
|7181| 2018|0.7558756868131872|11.667949158364825|     2912|
|7558| 2019|0.7018670886075954|11.308966244725742|     2844|
|7558| 2018| 0.739443099273608|11.813490141819441|     2891|
|7481| 2019|0.6953703703703707|13.503806584362135|     2916|
|7607| 2020|0.7647740757644912|16.179141944317678|     2197|
|7139| 2020|0.7439328426862939|14.197884084636618|     2175|
|7181| 2020|0.6911885245901641|14.058105646630224|     2197|
|7020| 2018|0.8082445987654319|12.319087452471484|     2840|
+----+-----+------------------+------------------+---------+
only showing top 10 rows



In [30]:
meteo.where('id < 8000')\
     .groupBy('id')\
     .agg(
        round(skewness  ('temperature'),3).alias('skewness'  ),
        round(kurtosis  ('temperature'),3).alias('kurtosis'  ),
        round(variance  ('temperature'),3).alias('variance'  ),
        round(var_pop   ('temperature'),3).alias('var_pop'   ),
        round(stddev    ('temperature'),3).alias('stddev'    ),
        round(stddev_pop('temperature'),3).alias('stddev_pop'))\
     .orderBy('id')\
     .show(15)

+----+--------+--------+--------+-------+------+----------+
|  id|skewness|kurtosis|variance|var_pop|stddev|stddev_pop|
+----+--------+--------+--------+-------+------+----------+
|7005|   0.066|  -0.042|  40.025| 40.024| 6.327|     6.326|
|7015|   0.122|   -0.19|  46.696| 46.695| 6.833|     6.833|
|7020|   -0.16|  -0.418|  17.011| 17.011| 4.124|     4.124|
|7027|   0.099|  -0.044|  36.238| 36.238|  6.02|      6.02|
|7037|   0.158|  -0.098|   44.12| 44.119| 6.642|     6.642|
|7072|   0.231|  -0.263|  56.528| 56.525| 7.518|     7.518|
|7110|    0.03|   0.138|  24.879| 24.879| 4.988|     4.988|
|7117|  -0.049|  -0.221|  18.651| 18.651| 4.319|     4.319|
|7130|   0.136|  -0.024|   41.93|  41.93| 6.475|     6.475|
|7139|   0.178|  -0.103|  46.922| 46.922|  6.85|      6.85|
|7149|   0.168|  -0.267|  52.845| 52.844| 7.269|     7.269|
|7168|   0.187|  -0.199|  59.685| 59.684| 7.726|     7.726|
|7181|   0.148|  -0.313|  60.995| 60.994|  7.81|      7.81|
|7190|   0.113|   -0.46|  67.557| 67.557

In [31]:
meteo.where('id < 8000 and annee > 2014')\
     .groupBy('id','annee')\
     .agg( round(avg('temperature'),2).alias('temperature'))\
     .orderBy("id","annee")\
     .show(10)

+----+-----+-----------+
|  id|annee|temperature|
+----+-----+-----------+
|7005| 2015|      11.27|
|7005| 2016|      10.92|
|7005| 2017|      11.46|
|7005| 2018|      11.56|
|7005| 2019|      11.51|
|7005| 2020|      13.77|
|7005| 2021|       11.0|
|7015| 2015|      11.38|
|7015| 2016|      11.16|
|7015| 2017|      11.66|
+----+-----+-----------+
only showing top 10 rows



# pivot
<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-04.png" width="400"> 

In [32]:
meteo.where('id < 8000 and annee > 2014')\
      .groupBy('id')\
      .pivot('annee')\
      .agg( round(avg('temperature'),2))\
      .sort('id')\
      .show(10)

+----+-----+-----+-----+-----+-----+-----+-----+
|  id| 2015| 2016| 2017| 2018| 2019| 2020| 2021|
+----+-----+-----+-----+-----+-----+-----+-----+
|7005|11.27|10.92|11.46|11.56|11.51|13.77| 11.0|
|7015|11.38|11.16|11.66|11.95|11.79| 14.3|11.26|
|7020|12.43|12.22|12.75|12.32|12.98|13.96|11.59|
|7027|11.57|11.15|11.71|11.64|11.58|13.68|11.24|
|7037| 11.2| 10.7|11.22|11.57|11.37|13.71| 10.8|
|7072|11.36|10.67|11.12|11.87|11.42|13.96| 11.3|
|7110|11.96|11.64|11.72|11.81|11.79| 13.5|11.45|
|7117|12.44|12.14|12.43|12.25|12.34|13.91|11.75|
|7130|12.36|11.82|12.18|12.65|12.43| 14.6|12.02|
|7139|11.49|10.94|11.51|11.91|11.71| 14.2|11.29|
+----+-----+-----+-----+-----+-----+-----+-----+
only showing top 10 rows



In [33]:
villes.select('ville',
               round('altitude',-2).alias('altitude'))\
      .groupBy('altitude')\
      .agg(collect_list('ville').alias('ville par altitude')).toPandas().head()
# .show(truncate=False)

,altitude,ville par altitude
0,300,"[NANCY-OCHEY, BALE-MULHOUSE, CLERMONT-FD, GOUR..."
1,900,[EMBRUN]
2,800,[LE PUY-LOUDES]
3,100,"[ABBEVILLE, CAEN-CARPIQUET, REIMS-PRUNAY, BRES..."
4,400,"[LIMOGES-BELLEGARDE, TARBES-OSSUN, ST GIRONS]"


In [34]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('pression'))\
     .orderBy('annee','mois')\
     .show(16)

+-----+----+--------+
|annee|mois|pression|
+-----+----+--------+
| 1997|   1|   982.0|
| 1997|   2|   890.0|
| 1997|   3|   985.0|
| 1997|   4|   951.0|
| 1997|   5|  1003.0|
| 1997|   6|   963.0|
| 1997|   7|   993.0|
| 1997|   8|   989.0|
| 1997|   9|   972.0|
| 1997|  10|   991.0|
| 1997|  11|   957.0|
| 1997|  12|   992.0|
| 1998|   1|   997.0|
| 1998|   2|   916.0|
| 1998|   3|  1011.0|
| 1998|   4|   955.0|
+-----+----+--------+
only showing top 16 rows

